Quick Analyse of 

as_con_3s.tif
as_msk_3s.tif
hyd_as_lup_15s.tif 


| File name              | Content / Purpose                                                             | Quick analysis idea                                      |
|------------------------|-------------------------------------------------------------------------------|---------------------------------------------------------|
| **as_con_3s.tif**      | Conditioned DEM — elevation model hydrologically corrected (stream burning, sinks filled) | Clip to Bhutan + buffer, plot hillshade, compare vs. raw DEM |
| **as_msk_3s.tif**      | Land mask — binary mask of land vs. ocean                                     | Clip to Bhutan + buffer, check unique values (0 = water, 1 = land) |
| **hyd_as_lup_15s.tif** | Lookup table at 15″ resolution (land/sea mask, auxiliary layer for hydrological atlas) | Clip to Bhutan extent, print unique values, verify categories |

In [ ]:
from pathlib import Path
import os
os.environ["GEOPANDAS_IO_ENGINE"] = "pyogrio"   
import geopandas as gpd
import rasterio   #This is a library for working with raster geospatial data (GeoTIFF)
import rioxarray as rxr # extension of xarray + rasterio, simplifies reading/writing GeoTIFF and reprojection to different CRS


con_path = Path("../../data/HydroSHEDS/as_con_3s.tif")
con_msk = Path("../../data/HydroSHEDS/as_msk_3s.tif")
hyd_as_lup = Path("../../data/HydroSHEDS/hyd_as_lup_15s.tif")

if not (con_path.exists() == con_msk.exists ()== hyd_as_lup.exists ()  == True):
    raise FileNotFoundError("One or more input files are missing. Please ensure all required files are present.")
else:
    print ("TODO")

FileNotFoundError: One or more input files are missing. Please ensure all required files are present.